In [111]:
import sys
import pandas as pd
import numpy as np 

sys.path.append('../')

In [112]:
ds_names = ["acl2020", "emnlp2018"]
tasks = ['fact', 'bias']
level = 0
ds_name = ds_names[1]
task = tasks[0]

df_edge = pd.read_csv('../data/edges_{}_level{}.txt'.format(ds_name, level))
df_nodes = pd.read_csv('../data/features_{}_level{}.txt'.format(ds_name, level))
df_label = pd.read_csv('../data/labels_{}_{}.csv'.format(ds_name, task))

nodes_dict=df_nodes.set_index('node').to_dict()
df_nodes = pd.merge(df_nodes, df_label, on='node', how='left')

# nodes_dict
# df_edge, df_nodes

In [113]:
print(len(edge_nodes), len(node_nodes))

3489 3489


In [114]:
len(edge_nodes - node_nodes)

0

In [115]:
len(node_nodes - edge_nodes)

0

In [116]:
common_nodes = edge_nodes & node_nodes
len(common_nodes)

3489

In [117]:
#added these to exclude mismatching node (needed to this for level 4 data of emnlp2018)
df_edge = df_edge[df_edge['source'].isin(common_nodes)]
df_edge = df_edge[df_edge['target'].isin(common_nodes)]
df_nodes = df_nodes[df_nodes['node'].isin(common_nodes)]

In [118]:
edge_nodes = set(df_edge['source'].unique()) | set(df_edge['target'].unique())
node_nodes = set(df_nodes['node'].unique())

In [119]:
node_idx = dict()
i = 0
for n in edge_nodes:
    #if n in common_nodes:
    node_idx[n] = i 
    i += 1
df_nodes['node_id'] = df_nodes['node'].apply(lambda x: node_idx[x] if x in node_idx else None )

In [120]:
#df_nodes = df_nodes[df_nodes['node_id'].notnull()]

In [121]:
next_val = len(df_nodes['label'].unique()) - 1
df_nodes['label'] = df_nodes['label'].fillna(next_val) #assign the next available number to unknown data points
df_nodes['label'] = df_nodes['label'].astype(int)
df_nodes['label'].value_counts()

3    2857
2     292
1     204
0     136
Name: label, dtype: int64

In [122]:
df_nodes['feat_domain'] = 1 #probably we should be able to exclude this

In [123]:
#two way edges
edge_df2 = pd.DataFrame()
edge_df2['source'] = df_edge['target']
edge_df2['target'] = df_edge['source']


df_edge2 = pd.concat([df_edge, edge_df2]).drop_duplicates(keep = "first").reset_index()
df_edge2['source'] = df_edge2['source'].apply(lambda x: node_idx[x])
df_edge2['target'] = df_edge2['target'].apply(lambda x: node_idx[x])

df_edge2.drop('index', axis=1, inplace=True)

df_nodes[['node_id', 'node', 'label',
         'alexa_rank', 'daily_pageviews_per_visitor',
         'daily_time_on_site', 'total_sites_linking_in', 'bounce_rate', 'feat_domain']].to_csv('../data/features_{}_level{}.txt2'.format(ds_name, level), index = False)
df_edge2.to_csv('../data/edges_{}_level{}.txt2'.format(ds_name, level), index = False)